#Model definition

In [ ]:
import numpy as np
import torch
import torch.nn as nn

In [ ]:
!git clone https://github.com/PepeCamposGarcia/PINNs.git

The problem we are aiming to solve is the Burgers equation with initial and boundary conditions:
$$
\begin{cases}
u_t + u  u_x - \left(\frac{0.01}{\pi}\right) u_{xx} = 0, & x \in [-1,1], \, t \in [0,1], \\
u(x, 0) = -\sin(\pi x),  \\
u(-1, t) = u(1, t) = 0,
\end{cases}
$$

In [ ]:
class SinusoidalActivation(nn.Module):
  def forward(self,x):
    return torch.sin(2 * np.pi * x)

In [ ]:
class Burgers1D(nn.Module):

############################### NET ARCHITECTURE ###############################

  def __init__(self, num_neurons, num_layers, nu):
    self.nu = nu

    super(Burgers1D, self).__init__()
    self.num_neurons = num_neurons
    self.num_layers = num_layers

### Imput layer, fully connected to the first hidden layer, sinusoidal activation
### function for the dismiss of local minima

    layer_list = [nn.Linear(2, self.num_neurons)]
    layer_list.append(SinusoidalActivation())

### Loop for the description of the hidden layers: fully connected layers,
### hiperbolic tangent activation function

    for _ in range (self.num_layers - 2):
      layer_list.append(nn.Linear(self.num_neurons, self.num_neurons))
      layer_list.append(nn.Tanh())

### Output layer, 1 output u

    layer_list.append(nn.Linear(self.num_neurons, 1))
    self.layers = nn.ModuleList(layer_list)


################## FEED-FORWARD PROPAGATION, OUTPUTS ###########################

  def forward(self, x, t):
     """
        Params:
        x - array of shape (N, 1), input x coordinates
        t - array of shape (N, 1), input time coordinate
        Returns:
        u - tensor of shape (N, 1), output x-velocity
        f - x-momentum PDE evaluation of shape (N, 1)

     """
     x = torch.tensor(x, dtype=torch.float32, requires_grad=True)
     t = torch.tensor(t, dtype=torch.float32, requires_grad=True)

     input_data = torch.hstack([x, t])

     out = input_data

     for layer in self.layers:
      out = layer(out)

     u = out[:] # (N, 1) each

     u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]
     u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
     u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]

        # Evaluate momentum PDE's

     f = u_t + u * u_x - self.nu * u_xx # (N, 1)


     return u, f

#Loss function

In [ ]:
##################### LOSS FUNCTION DESIGN #####################################

class Burgers1DLoss(nn.Module):
  def __init__(self):
    super().__init__()
    self.mse = nn.MSELoss()

  def forward(self, u_init, u_net_init, u_net_BC, f):

     L_init = self.mse(u_init, u_net_init)
     L_boundary = self.mse(u_net_BC, torch.zeros_like(u_net_BC))
     L_pde  = self.mse(f, torch.zeros_like(f))

     return L_init + L_boundary + L_pde

#Data loading

In [ ]:
import scipy

In [ ]:
def boundary_indices(T, N):
    """
    Returns a boolean mask marking the boundary spatial points for all timesteps
    Params:
    N - # of data samples (space locations) in problem
    T - # of timesteps
    Return:
    nd-array of shape (N*T, )
    """
    # Create grid for one timestep
    # Data is a 100 x 256 matrix
    # This function creates a boolean mask to identify every point with x=1 or x=-1


    grid = np.zeros((100, 256))

    # Set boundary to 1
    grid[:, 0] = 1
    grid[:, -1] = 1

    grid_in = grid.flatten().reshape(-1,1)

    # Flatten final grid into column vector to be used in training
    boundary_positions = grid_in.astype(bool)

    return boundary_positions


In [ ]:
def init_indices(T, N):
    """
    Returns a boolean mask marking the initial condition points
    Params:
    N - # of data samples (space locations) in problem
    T - # of timesteps
    Return:
    nd-array of shape (N*T, )
    """
    # Create grid for one timestep
    # Data consists of a 100 x 256 matrix
    # This function creates a boolean mask to identify every point with t=0


    grid = np.zeros((100, 256))

    # Set boundary to 1

    grid[0, :] = 1

    grid_init_in = grid.flatten().reshape(-1,1)

    # Flatten final grid into column vector to be used in training
    init_positions = grid_init_in.astype(bool)

    return init_positions


In [ ]:
import csv

In [ ]:
def data_loading () :
  x=torch.linspace(-1,1,256)
  t=torch.linspace(0,1,100)
  N, T = x.shape[0], t.shape[0]
  XX,TT = torch.meshgrid(x,t)
  XX = XX.transpose(1,0)
  TT = TT.transpose(1,0)

  x_in = XX.flatten().reshape(-1, 1)  # NT x 1
  t_in = TT.flatten().reshape(-1, 1)  # NT x 1

  return x, t, XX, TT, x_in, t_in, (T, N)

#Training set

In [ ]:
x, t, XX, TT, x_in, t_in, (T, N) = data_loading () # (NT, 1)
# For boundary an initial points, it is in the variables idx_b_train and
# idx_i_train where we choose how many of them will be used in training,
# in this case 50 of each are selected

# Boundary training points
idx_b = boundary_indices(T, N)
x_boundary = x_in[idx_b, :]
t_boundary = t_in[idx_b, :]
idx_b_train = np.random.choice(x_boundary.shape[0], 50, replace=False)
x_b_train = x_boundary[idx_b_train].reshape(-1,1)
t_b_train = t_boundary[idx_b_train].reshape(-1,1)


# Initial training points
idx_i = init_indices(T ,N)
x_init = x_in[idx_i, :]
idx_i_train = np.random.choice(x_init.shape[0], 50, replace=False)
x_i_train = x_init[idx_i_train].reshape(-1,1)
t_i_train = torch.zeros_like(x_i_train).reshape(-1,1)

u_init = -torch.sin(torch.pi*x_i_train)
u_init = torch.tensor(u_init, dtype = torch.float32)
u_init = u_init.reshape(-1,1)


# Collocation points

# Choose fraction of the total points to be used in training set as collocation points.
# In this case a 45% of the total 26500 are used
samples = int(round(N * T * 0.45))

idx = np.random.choice(x_in.shape[0], samples, replace=False)
x_train = x_in[idx, :]
t_train = t_in[idx, :]

#Training algorithm

In [ ]:
from tqdm import tqdm

In [ ]:
def main(num_neurons, num_layers, epochs):
    """
    Params:
    num_neurons - int, # of hidden units for each neural network layer
    num_layers - int, # of neural network layers
    epochs - int, # of training epochs
    train_selection - float (0,1), fraction of all data (N*T) to select for training OR
                      'BC', select the boundary conditions for training (all timesteps)
    """
    nu = 0.01/torch.pi


    PINN_model = Burgers1D(num_neurons=num_neurons, num_layers=num_layers, nu=nu)
    criterion = Burgers1DLoss()

    optimizer = torch.optim.LBFGS(PINN_model.parameters(), line_search_fn='strong_wolfe')

    def closure():
        """Define closure function to use with LBFGS optimizer"""
        optimizer.zero_grad()   # Clear gradients from previous iteration

        u_net, f = PINN_model(x_train, t_train)
        u_net_BC, g = PINN_model(x_b_train, t_b_train)
        u_net_init, h = PINN_model(x_i_train, t_i_train)

        loss = criterion(u_init, u_net_init, u_net_BC, f)

        loss.backward() # Backprogation
        return loss

    def training_loop(epochs):
        """Run training loop"""


        for i in tqdm(range(epochs), desc='Training epochs: '):


          optimizer.step(closure)
          loss = closure().item()

    training_loop(epochs=epochs)
    torch.save(PINN_model.state_dict(), 'model_name.pth')
    return
    print(x_train.shape)



Running the following cell will start the network training, and generate a .pth file named 'model_name.pth', with the width and depth specified in 'num_neurons' and 'num_layers', and trained over 'epochs' iterations. The training set used was constructed in the previous section.

In [ ]:
if __name__ == '__main__':
    main(num_neurons=num_neurons, num_layers=num_layers, epochs=epochs)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd

#Different architecture models. Model loading
In the next section, different architecture networks are trained on the same number of iterations to analyse how  width and depth of the model impact the accuracy of the predicted solution compared to the exact one.

10 NEURONS

In [ ]:
def load_saved_model_10_3(num_layers, num_neurons):

    nu = 0.01

    # load saved state_dict
    PINN_model = Burgers1D(num_neurons=num_neurons, num_layers=num_layers, nu=nu)
    PINN_model.load_state_dict(torch.load('/content/PINNs/Models_Data_Figures/BURGERS/Burgers_Architecture/Models/10_3.pth'))
    PINN_model.eval() # Set model to evaluation mode
    return PINN_model

In [ ]:
def load_saved_model_10_5(num_layers, num_neurons):

    nu = 0.01

    # load saved state_dict
    PINN_model = Burgers1D(num_neurons=num_neurons, num_layers=num_layers, nu=nu)
    PINN_model.load_state_dict(torch.load('/content/PINNs/Models_Data_Figures/BURGERS/Burgers_Architecture/Models/10_5.pth'))
    PINN_model.eval() # Set model to evaluation mode
    return PINN_model

In [ ]:
def load_saved_model_10_7(num_layers, num_neurons):

    nu = 0.01

    # load saved state_dict
    PINN_model = Burgers1D(num_neurons=num_neurons, num_layers=num_layers, nu=nu)
    PINN_model.load_state_dict(torch.load('/content/PINNs/Models_Data_Figures/BURGERS/Burgers_Architecture/Models/10_7.pth'))
    PINN_model.eval() # Set model to evaluation mode
    return PINN_model

In [ ]:
def load_saved_model_10_9(num_layers, num_neurons):

    nu = 0.01

    # load saved state_dict
    PINN_model = Burgers1D(num_neurons=num_neurons, num_layers=num_layers, nu=nu)
    PINN_model.load_state_dict(torch.load('/content/PINNs/Models_Data_Figures/BURGERS/Burgers_Architecture/Models/10_9.pth'))
    PINN_model.eval() # Set model to evaluation mode
    return PINN_model

20 NEURONS

In [ ]:
def load_saved_model_20_3(num_layers, num_neurons):

    nu = 0.01

    # load saved state_dict
    PINN_model = Burgers1D(num_neurons=num_neurons, num_layers=num_layers, nu=nu)
    PINN_model.load_state_dict(torch.load('/content/PINNs/Models_Data_Figures/BURGERS/Burgers_Architecture/Models/20_3.pth'))
    PINN_model.eval() # Set model to evaluation mode
    return PINN_model

In [ ]:
def load_saved_model_20_5(num_layers, num_neurons):

    nu = 0.01

    # load saved state_dict
    PINN_model = Burgers1D(num_neurons=num_neurons, num_layers=num_layers, nu=nu)
    PINN_model.load_state_dict(torch.load('/content/PINNs/Models_Data_Figures/BURGERS/Burgers_Architecture/Models/20_5.pth'))
    PINN_model.eval() # Set model to evaluation mode
    return PINN_model

In [ ]:
def load_saved_model_20_7(num_layers, num_neurons):

    nu = 0.01

    # load saved state_dict
    PINN_model = Burgers1D(num_neurons=num_neurons, num_layers=num_layers, nu=nu)
    PINN_model.load_state_dict(torch.load('/content/PINNs/Models_Data_Figures/BURGERS/Burgers_Architecture/Models/20_7.pth'))
    PINN_model.eval() # Set model to evaluation mode
    return PINN_model

In [ ]:
def load_saved_model_20_9(num_layers, num_neurons):

    nu = 0.01

    # load saved state_dict
    PINN_model = Burgers1D(num_neurons=num_neurons, num_layers=num_layers, nu=nu)
    PINN_model.load_state_dict(torch.load('/content/PINNs/Models_Data_Figures/BURGERS/Burgers_Architecture/Models/20_9.pth'))
    PINN_model.eval() # Set model to evaluation mode
    return PINN_model

40 NEURONS

In [ ]:
def load_saved_model_40_3(num_layers, num_neurons):

    nu = 0.01

    # load saved state_dict
    PINN_model = Burgers1D(num_neurons=num_neurons, num_layers=num_layers, nu=nu)
    PINN_model.load_state_dict(torch.load('/content/PINNs/Models_Data_Figures/BURGERS/Burgers_Architecture/Models/40_3.pth'))
    PINN_model.eval() # Set model to evaluation mode
    return PINN_model

In [ ]:
def load_saved_model_40_5(num_layers, num_neurons):

    nu = 0.01

    # load saved state_dict
    PINN_model = Burgers1D(num_neurons=num_neurons, num_layers=num_layers, nu=nu)
    PINN_model.load_state_dict(torch.load('/content/PINNs/Models_Data_Figures/BURGERS/Burgers_Architecture/Models/40_5.pth'))
    PINN_model.eval() # Set model to evaluation mode
    return PINN_model

In [ ]:
def load_saved_model_40_7(num_layers, num_neurons):

    nu = 0.01

    # load saved state_dict
    PINN_model = Burgers1D(num_neurons=num_neurons, num_layers=num_layers, nu=nu)
    PINN_model.load_state_dict(torch.load('/content/PINNs/Models_Data_Figures/BURGERS/Burgers_Architecture/Models/40_7.pth'))
    PINN_model.eval() # Set model to evaluation mode
    return PINN_model

In [ ]:
def load_saved_model_40_9(num_layers, num_neurons):

    nu = 0.01

    # load saved state_dict
    PINN_model = Burgers1D(num_neurons=num_neurons, num_layers=num_layers, nu=nu)
    PINN_model.load_state_dict(torch.load('/content/PINNs/Models_Data_Figures/BURGERS/Burgers_Architecture/Models/40_9.pth'))
    PINN_model.eval() # Set model to evaluation mode
    return PINN_model

MODELS

In [ ]:
model_10_3 = load_saved_model_10_3(num_layers = 3, num_neurons = 10)
model_10_5 = load_saved_model_10_5(num_layers = 5, num_neurons = 10)
model_10_7 = load_saved_model_10_7(num_layers = 7, num_neurons = 10)
model_10_9 = load_saved_model_10_9(num_layers = 9, num_neurons = 10)

In [ ]:
model_20_3 = load_saved_model_20_3(num_layers = 3, num_neurons = 20)
model_20_5 = load_saved_model_20_5(num_layers = 5, num_neurons = 20)
model_20_7 = load_saved_model_20_7(num_layers = 7, num_neurons = 20)
model_20_9 = load_saved_model_20_9(num_layers = 9, num_neurons = 20)

In [ ]:
model_40_3 = load_saved_model_40_3(num_layers = 3, num_neurons = 40)
model_40_5 = load_saved_model_40_5(num_layers = 5, num_neurons = 40)
model_40_7 = load_saved_model_40_7(num_layers = 7, num_neurons = 40)
model_40_9 = load_saved_model_40_9(num_layers = 9, num_neurons = 40)

#Model evaluation

In [ ]:
#MODEL EVALUATION

#u_n1_n2 is a tensor containing the predicted solution across the spatio-temporal domain
#f_n1_n2 is a tensor containing the evaluation of the Burgers equation
#UU_n1_n2 is a reshape of the solution for its representation
#n1 refers to the number of neurons per layer of the model
#n2 refers to the number of layers of the model

# 10 neurons
u_10_3, f_10_3 = model_10_3(x_in, t_in)
UU_10_3 = u_10_3.reshape(T, N).detach().transpose(1,0)

u_10_5, f_10_5 = model_10_5(x_in, t_in)
UU_10_5 = u_10_5.reshape(T, N).detach().transpose(1,0)

u_10_7, f_10_7 = model_10_7(x_in, t_in)
UU_10_7 = u_10_7.reshape(T, N).detach().transpose(1,0)

u_10_9, f_10_9 = model_10_9(x_in, t_in)
UU_10_9 = u_10_9.reshape(T, N).detach().transpose(1,0)

#20 neurons
u_20_3, f_20_3 = model_20_3(x_in, t_in)
UU_20_3 = u_20_3.reshape(T, N).detach().transpose(1,0)

u_20_5, f_20_5 = model_20_5(x_in, t_in)
UU_20_5 = u_20_5.reshape(T, N).detach().transpose(1,0)

u_20_7, f_20_7 = model_20_7(x_in, t_in)
UU_20_7 = u_20_7.reshape(T, N).detach().transpose(1,0)

u_20_9, f_20_9 = model_20_9(x_in, t_in)
UU_20_9 = u_20_9.reshape(T, N).detach().transpose(1,0)

#40 neurons
u_40_3, f_40_3 = model_40_3(x_in, t_in)
UU_40_3 = u_40_3.reshape(T, N).detach().transpose(1,0)

u_40_5, f_40_5 = model_40_5(x_in, t_in)
UU_40_5 = u_40_5.reshape(T, N).detach().transpose(1,0)

u_40_7, f_40_7 = model_40_7(x_in, t_in)
UU_40_7 = u_40_7.reshape(T, N).detach().transpose(1,0)

u_40_9, f_40_9 = model_40_9(x_in, t_in)
UU_40_9 = u_40_9.reshape(T, N).detach().transpose(1,0)


In [ ]:
XX_p=XX.detach().transpose(1,0)
TT_p=TT.detach().transpose(1,0)

#Exact solution

In [ ]:
def data_loading_2 ():

  data = scipy.io.loadmat('/content/PINNs/Models_Data_Figures/Data/burgers_equation.mat')
  UU_star = torch.tensor(data['usol']) #        256 x 100
  X_star = data['x']                   #        256 x 1
  t_star = data['t']                   #        100  x 1

  N, T= X_star.shape[0], t_star.shape[0]

  # Column vector transformation  25600 x 1
  u_star_in = UU_star.flatten().reshape(-1, 1)

  return UU_star, u_star_in

In [ ]:
UU_star, u_star_in = data_loading_2 ()

#Root mean squared error

In [ ]:
def rMSE(UU):

    # Initialize array to hold sum of square errors (sse)
    N = UU.shape[0]
    T = UU.shape[1]

    sse = np.zeros(T)

    for i in range(T):
        # Prediction for time i
        sse[i] = ((UU[:,i]-UU_star[:,i])**2).sum()

    # Sum sse over timesteps
    sse = sse.sum()

    # Average sse over all samples and all timesteps to get mean squared error (mse)
    mse = sse / (N * T)

    #R^2 coefficient calculation
    # Find total sum of squares (SSTO)
    ssto = ((u_star_in - u_star_in.mean()) ** 2).sum()
    ssto = ssto.detach()
    R2 = (1 - (sse / ssto))

    return mse, R2.detach()

In [ ]:
# Calculate rmse
Layer_list_10 = [3, 5, 7, 9]
U_list_10 = [UU_10_3, UU_10_5, UU_10_7, UU_10_9]

rmse_list_10, r2_list_10 = [], []
for i, layer in enumerate(Layer_list_10):
    rmse_i, r2_i = rMSE(U_list_10[i]) #each of shape (1, )
    rmse_list_10.append(rmse_i)
    r2_list_10.append(r2_i)

rmse_10, r2_10 = np.vstack(rmse_list_10), np.vstack(r2_list_10) # # epochs tested, u
rmse_df_10, r2_df_10 = pd.DataFrame(rmse_10, columns=['10 neurons per layer'], index=Layer_list_10), pd.DataFrame(r2_10, columns=['10 neurons per layer'], index=Layer_list_10)

In [ ]:
# Calculate rmse
Layer_list_20 = [3, 5, 7, 9]
U_list_20 = [UU_20_3, UU_20_5, UU_20_7, UU_20_9]

rmse_list_20, r2_list_20 = [], []
for i, epoch in enumerate(Layer_list_20):
    rmse_i, r2_i = rMSE(U_list_20[i]) #each of shape (1, )
    rmse_list_20.append(rmse_i)
    r2_list_20.append(r2_i)

rmse_20, r2_20 = np.vstack(rmse_list_20), np.vstack(r2_list_20) # # epochs tested, u
rmse_df_20, r2_df_20 = pd.DataFrame(rmse_20, columns=['20 neurons per layer'], index=Layer_list_20), pd.DataFrame(r2_20, columns=['20 neurons per layer'], index=Layer_list_20)

In [ ]:
# Calculate rmse
Layer_list_40 = [3, 5, 7, 9]
U_list_40 = [UU_40_3, UU_40_5, UU_40_7, UU_40_9]

rmse_list_40, r2_list_40 = [], []
for i, epoch in enumerate(Layer_list_40):
    rmse_i, r2_i = rMSE(U_list_40[i]) #each of shape (1, )
    rmse_list_40.append(rmse_i)
    r2_list_40.append(r2_i)

rmse_40, r2_40 = np.vstack(rmse_list_40), np.vstack(r2_list_40) # # epochs tested, u
rmse_df_40, r2_df_40 = pd.DataFrame(rmse_40, columns=['40 neurons per layer'], index=Layer_list_40), pd.DataFrame(r2_40, columns=['40 neurons per layer'], index=Layer_list_40)

#Results table

The following table summarizes the results obtained during this section, reflecting the predictions' root mean squared error with respect to the exact solution across all the domain. A more complex architecture leads to more accurate results.

In [ ]:
df_rmse = pd.concat([rmse_df_10, rmse_df_20, rmse_df_40], axis=1)
df_rmse

,10 neurons per layer,20 neurons per layer,40 neurons per layer
3,0.078619,0.036911,0.015361
5,0.015296,0.000475,0.000039
7,0.003723,0.000646,0.000027
9,0.000231,0.000157,0.000024


In [ ]:
import pandas as pd
from google.colab import files

In [ ]:
df_rmse.to_csv('estudioarquitecturadef.csv', index=True)
files.download('estudioarquitecturadef.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>